Función Creada por Leon para la carga de los archivos .mat sigue la linea que uso en el Notebook Extracción de Señal Prueba

In [21]:
import scipy.io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

In [22]:
def load_emg(file_path):
    """
    Carga un archivo .mat de LabChart y extrae los canales de Biceps y Triceps.

    Parámetros
    ----------
    file_path : str o Path
        Ruta relativa o absoluta al archivo .mat

    Retorna
    -------
    dict con claves 'Biceps' y 'Triceps' que contienen los arrays de señal
    """
    data = scipy.io.loadmat(file_path)

    # Extraemos la señal principal
    signals = data["data"].squeeze()

    # Extraemos títulos de los canales
    titles = data["titles"].squeeze()

    # Creamos un diccionario con las señales que necesitamos
    result = {}
    for i, title in enumerate(titles):
        if "Biceps" in title or "Triceps" in title:
            start = int(data["datastart"][i, 0]) - 1
            end = int(data["dataend"][i, 0])
            result[title.strip()] = signals[start:end]

##Debido a que le vamos a dar de entrada un dataframe a window slicing hay que anadir un vector de tiempo (Pipe)
    samplerate = float(data["samplerate"].squeeze()[2]) #se usa 2 por que 0 y 1 estan vacios

    # Calcular el vector de tiempo
    # Tomamos la longitud de cualquiera de las señales (ambas tienen misma duración)
    if "Biceps" in result:
        n_samples = len(result["Biceps"])
    else:
        # Si no hay Biceps, usamos Triceps
        n_samples = len(result["Triceps"])

    duration = n_samples / samplerate
    # Vector de 0 a duración, con paso 1/samplerate
    time_vector = np.arange(0, duration, 1.0 / samplerate)

    # Aseguramos que la longitud coincida exactamente
    time_vector = time_vector[:n_samples]

    result["tiempo"] = time_vector
    db=pd.DataFrame(data=result)

    return db

In [23]:
signal=load_emg('SEÑALES EMG\\SUJETO1\\M1_JUAN_VANEGAS.mat')
print(signal)

             Biceps   Triceps   tiempo
0     -3.125000e-07 -0.000002   0.0000
1     -4.562500e-06 -0.000026   0.0005
2     -1.912500e-05 -0.000103   0.0010
3     -4.781250e-05 -0.000241   0.0015
4     -8.790625e-05 -0.000414   0.0020
...             ...       ...      ...
68384 -2.255937e-04 -0.000803  34.1920
68385 -2.342500e-04 -0.000805  34.1925
68386 -2.331250e-04 -0.000776  34.1930
68387 -2.238750e-04 -0.000723  34.1935
68388 -2.074688e-04 -0.000647  34.1940

[68389 rows x 3 columns]


Añadí (Pipe) cosas a la función para tener el dataframe (no creo que el orden de las columnas importe por ahora)